In [1]:
import pandas as pd
import datetime as dt

In [2]:
bbbcov = pd.read_csv('476_bbbcov_afterRRupload_apr2023.csv',sep=',')

In [3]:
bbbcov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   MR ID    48 non-null     object 
 1   Date     48 non-null     object 
 2   Subject  48 non-null     object 
 3   Age      0 non-null      float64
 4   Scanner  45 non-null     object 
 5   Scans    48 non-null     object 
dtypes: float64(1), object(5)
memory usage: 2.4+ KB


In [4]:
xnatdate = bbbcov['Date']
xnatdate = xnatdate.apply(dt.datetime.fromisoformat)
bbbcov = bbbcov.drop('Date', axis='columns')
bbbcov = bbbcov.drop('Age', axis='columns')
bbbcov = bbbcov.drop('Scanner', axis='columns')
bbbcov['Date'] = xnatdate

In [5]:
bbbcov.style

,MR ID,Subject,Scans,Date
0,22_03_02-15_39_22-STD-1_3_12_2_1107_5_2_34_18984,240616-301,"3D_localizer_v2_200V(1), (1), (1), MB_GRE_EPI_400VOL_RS(1), MB_GRE_EPI_400VOL_RS_SBRef(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_UNI-DEN(1), Quick_Tx_calib_200V_brain(2), Quick_Tx_calib_200V_pons(4), gre_NoiseScans_A32_withNoise_200V(2), gre_NoiseScans_V32_noNoise_200V(2), gre_NoiseScans_V32_noNoise_200V_Fitted_Sens(2), gre_NoiseScans_V32_noNoise_200V_Raw_Sens(2), localizer(1), localizer_200V(1)",2022-03-02 00:00:00
1,22_04_01-12_03_22-DST-1_3_12_2_1107_5_2_34_18984,251113-7,"3D_localizer_v2_200V(1), (1), (1), MB_GRE_EPI_400VOL_RS(1), MB_GRE_EPI_400VOL_RS_SBRef(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_UNI-DEN(1), Quick_Tx_calib_200V_pons(2), gre_NoiseScans_A32_withNoise_200V(2), gre_NoiseScans_V32_noNoise_200V(2), gre_NoiseScans_V32_noNoise_200V_Fitted_Sens(2), gre_NoiseScans_V32_noNoise_200V_Raw_Sens(2), gre_field_mapping_2mm_resting(2), localizer_200V(1), sLaser_Brainstem_20_12_12(1), sLaser_Lw(1), sLaser_WS_Cal(2)",2022-04-01 00:00:00
2,22_05_06-11_48_16-DST-1_3_12_2_1107_5_2_34_18984,CAU002N001,"3D_localizer_v2_200V(1), (1), (1), MB_GRE_EPI_400VOL_RS(1), MB_GRE_EPI_400VOL_RS_SBRef(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_RR_UniformSens_MAG(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_RR_UniformSens_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_UNI-DEN(1), Quick_Tx_calib_200V_pons(2), T2star_0.7_6echo_UniformNoise_MAG(1), T2star_0.7_6echo_UniformNoise_PHS(1), T2star_0.7_6echo_UniformSens_MAG(1), T2star_0.7_6echo_UniformSens_PHS(1), gre_NoiseScans_A32_withNoise_200V(2), gre_NoiseScans_V32_noNoise_200V(2), gre_NoiseScans_V32_noNoise_200V_Fitted_Sens(2), gre_NoiseScans_V32_noNoise_200V_Raw_Sens(2), gre_field_mapping_2mm_resting(4), localizer_200V(1), sLaser_Brainstem_20_12_12(3), sLaser_Brainstem_20_12_12_OFF(1), sLaser_Brainstem_20_12_12_RF_OFF(1), sLaser_Lw(1), sLaser_WS_Cal(1)",2022-05-06 00:00:00
3,22_05_20-11_27_39-DST-1_3_12_2_1107_5_2_34_18984,CAU002N002,"3D_localizer_v2_200V(1), (1), (1), MB_GRE_EPI_400VOL_RS(1), MB_GRE_EPI_400VOL_RS_SBRef(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_RR_UniformSens_MAG(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_RR_UniformSens_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_UNI-DEN(1), Quick_Tx_calib_200V_pons(2), T2star_0.7_6echo_UniformNoise_MAG(1), T2star_0.7_6echo_UniformNoise_PHS(1), T2star_0.7_6echo_UniformSens_MAG(1), T2star_0.7_6echo_UniformSens_PHS(1), gre_NoiseScans_A32_withNoise_200V(2), gre_NoiseScans_V32_noNoise_200V(2), gre_NoiseScans_V32_noNoise_200V_Fitted_Sens(2), gre_NoiseScans_V32_noNoise_200V_Raw_Sens(2), gre_field_mapping_2mm_resting(2), localizer_200V(1), sLaser_Brainstem_20_12_12(1), sLaser_Brainstem_20_12_12_OFF(1), sLaser_Brainstem_20_12_12_RF_OFF(1), sLaser_Lw(9), sLaser_WS_Cal(1)",2022-05-20 00:00:00
4,22_06_06-10_54_15-DST-1_3_12_2_1107_5_2_34_18984,CAU002N004,"3D_localizer_1mm_200V(1), 3D_localizer_v2_200V(1), (1), (1), MB_GRE_EPI_400VOL_RS(1), MB_GRE_EPI_400VOL_RS_SBRef(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_RR_U

In [6]:
# drop duplicates
bbbcov = bbbcov.drop([29,37,40], axis='index')

In [7]:
# bool Series of participants, filtering typos and pilots 
isppt=bbbcov['Subject'].str.contains('CAU002N0')
bbb = bbbcov.loc[isppt]
bbb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 2 to 44
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   MR ID    39 non-null     object        
 1   Subject  39 non-null     object        
 2   Scans    39 non-null     object        
 3   Date     39 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 1.5+ KB


In [8]:
bbb[['MR ID','Subject','Date']].style

,MR ID,Subject,Date
2,22_05_06-11_48_16-DST-1_3_12_2_1107_5_2_34_18984,CAU002N001,2022-05-06 00:00:00
3,22_05_20-11_27_39-DST-1_3_12_2_1107_5_2_34_18984,CAU002N002,2022-05-20 00:00:00
4,22_06_06-10_54_15-DST-1_3_12_2_1107_5_2_34_18984,CAU002N004,2022-06-06 00:00:00
5,22_06_17-11_24_30-DST-1_3_12_2_1107_5_2_34_18984,CAU002N006,2022-06-17 00:00:00
6,22_06_20-11_04_52-DST-1_3_12_2_1107_5_2_34_18984,CAU002N007,2022-06-20 00:00:00
7,22_06_24-11_04_56-DST-1_3_12_2_1107_5_2_34_18984,CAU002N008,2022-06-24 00:00:00
8,22_06_27-10_55_36-DST-1_3_12_2_1107_5_2_34_18984,CAU002N009,2022-06-27 00:00:00
9,22_07_01-10_58_54-DST-1_3_12_2_1107_5_2_34_18984,CAU002N010,2022-07-01 00:00:00
10,22_07_08-09_37_52-DST-1_3_12_2_1107_5_2_34_18984,CAU002N011,2022-07-08 00:00:00
11,22_07_11-10_03_51-DST-1_3_12_2_1107_5_2_34_18984,CAU002N012,2022-07-11 00:00:00


In [9]:
# check whether any useful data in scan (if no MP2RAGE, didn't get very far)
is_mp2rage=bbb['Scans'].str.contains('MP2RAGE_UK7T')
is_mp2rage.value_counts()

True     37
False     2
Name: Scans, dtype: int64

In [10]:
bbb[is_mp2rage].style

,MR ID,Subject,Scans,Date
2,22_05_06-11_48_16-DST-1_3_12_2_1107_5_2_34_18984,CAU002N001,"3D_localizer_v2_200V(1), (1), (1), MB_GRE_EPI_400VOL_RS(1), MB_GRE_EPI_400VOL_RS_SBRef(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_RR_UniformSens_MAG(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_RR_UniformSens_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_UNI-DEN(1), Quick_Tx_calib_200V_pons(2), T2star_0.7_6echo_UniformNoise_MAG(1), T2star_0.7_6echo_UniformNoise_PHS(1), T2star_0.7_6echo_UniformSens_MAG(1), T2star_0.7_6echo_UniformSens_PHS(1), gre_NoiseScans_A32_withNoise_200V(2), gre_NoiseScans_V32_noNoise_200V(2), gre_NoiseScans_V32_noNoise_200V_Fitted_Sens(2), gre_NoiseScans_V32_noNoise_200V_Raw_Sens(2), gre_field_mapping_2mm_resting(4), localizer_200V(1), sLaser_Brainstem_20_12_12(3), sLaser_Brainstem_20_12_12_OFF(1), sLaser_Brainstem_20_12_12_RF_OFF(1), sLaser_Lw(1), sLaser_WS_Cal(1)",2022-05-06 00:00:00
3,22_05_20-11_27_39-DST-1_3_12_2_1107_5_2_34_18984,CAU002N002,"3D_localizer_v2_200V(1), (1), (1), MB_GRE_EPI_400VOL_RS(1), MB_GRE_EPI_400VOL_RS_SBRef(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_RR_UniformSens_MAG(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_RR_UniformSens_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_UNI-DEN(1), Quick_Tx_calib_200V_pons(2), T2star_0.7_6echo_UniformNoise_MAG(1), T2star_0.7_6echo_UniformNoise_PHS(1), T2star_0.7_6echo_UniformSens_MAG(1), T2star_0.7_6echo_UniformSens_PHS(1), gre_NoiseScans_A32_withNoise_200V(2), gre_NoiseScans_V32_noNoise_200V(2), gre_NoiseScans_V32_noNoise_200V_Fitted_Sens(2), gre_NoiseScans_V32_noNoise_200V_Raw_Sens(2), gre_field_mapping_2mm_resting(2), localizer_200V(1), sLaser_Brainstem_20_12_12(1), sLaser_Brainstem_20_12_12_OFF(1), sLaser_Brainstem_20_12_12_RF_OFF(1), sLaser_Lw(9), sLaser_WS_Cal(1)",2022-05-20 00:00:00
4,22_06_06-10_54_15-DST-1_3_12_2_1107_5_2_34_18984,CAU002N004,"3D_localizer_1mm_200V(1), 3D_localizer_v2_200V(1), (1), (1), MB_GRE_EPI_400VOL_RS(1), MB_GRE_EPI_400VOL_RS_SBRef(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_RR_UniformSens_MAG(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_RR_UniformSens_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_UNI-DEN(1), Quick_Tx_calib_200V_pons(2), T2star_0.7_6echo_UniformNoise_MAG(1), T2star_0.7_6echo_UniformNoise_PHS(1), T2star_0.7_6echo_UniformSens_MAG(1), T2star_0.7_6echo_UniformSens_PHS(1), gre_NoiseScans_A32_withNoise_200V(2), gre_NoiseScans_V32_noNoise_200V(2), gre_NoiseScans_V32_noNoise_200V_Fitted_Sens(2), gre_NoiseScans_V32_noNoise_200V_Raw_Sens(2), gre_field_mapping_2mm_resting(2), localizer_200V(1), sLaser_Brainstem_20_12_12(1), sLaser_Brainstem_20_12_12_OFF(1), sLaser_Brainstem_20_12_12_RF_OFF(1), sLaser_Lw(2), sLaser_WS_Cal(1)",2022-06-06 00:00:00
5,22_06_17-11_24_30-DST-1_3_12_2_1107_5_2_34_18984,CAU002N006,"3D_localizer_1mm_200V(1), 3D_localizer_v2_200V(1), (1), (1), MB_GRE_EPI_400VOL_RS(1), MB_GRE_EPI_400VOL_RS_SBRef(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV1_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_INV2_PHS(1), MP2RAGE_UK7T_081018_tfl_wip944_b17stx_UNI-DEN(1), Quick_Tx_calib_200V_pons(2), T2star_0.7_6echo_UniformNoise_MAG(1), T2star_0.7_6echo_UniformNoise_PHS(1), T2star_0.7_6echo_UniformSens_MAG(1), T2star_0.7_6echo_UniformSens_PHS(1), gre_NoiseScans_A32_withNoise_200V(2), gre_NoiseScans_V32_noNoise_200V(2), gre_NoiseScans_V32_noNoise_200V_Fitted_Sens(2),

In [11]:
is_mrs=bbb['Scans'].str.contains('Brainstem')
is_mrs.value_counts()

True     37
False     2
Name: Scans, dtype: int64

In [19]:
bbb[~is_mrs]

,MR ID,Subject,Scans,Date
24,22_09_26-11_09_05-DST-1_3_12_2_1107_5_2_34_18984,CAU002N027,"3D_localizer_v2_200V(1), <MPR Range>(1), <MPR ...",2022-09-26
25,22_09_30-11_25_07-DST-1_3_12_2_1107_5_2_34_18984,CAU002N028,"3D_localizer_1mm_200V(1), Quick_Tx_calib_200V_...",2022-09-30


In [12]:
is_fmri=bbb['Scans'].str.contains('MB_GRE_EPI_400VOL_RS')
is_fmri.value_counts()

True     37
False     2
Name: Scans, dtype: int64

In [20]:
bbb[~is_fmri]

,MR ID,Subject,Scans,Date
24,22_09_26-11_09_05-DST-1_3_12_2_1107_5_2_34_18984,CAU002N027,"3D_localizer_v2_200V(1), <MPR Range>(1), <MPR ...",2022-09-26
25,22_09_30-11_25_07-DST-1_3_12_2_1107_5_2_34_18984,CAU002N028,"3D_localizer_1mm_200V(1), Quick_Tx_calib_200V_...",2022-09-30


In [13]:
is_t2star=bbb['Scans'].str.contains('T2star_0.7_6echo_UniformSens')
is_t2star.value_counts()

True     36
False     3
Name: Scans, dtype: int64

In [21]:
bbb[~is_t2star]

,MR ID,Subject,Scans,Date
19,22_08_19-11_24_59-DST-1_3_12_2_1107_5_2_34_18984,CAU002N020,"3D_localizer_1mm_200V(1), <MPR Range>(1), <MPR...",2022-08-19
24,22_09_26-11_09_05-DST-1_3_12_2_1107_5_2_34_18984,CAU002N027,"3D_localizer_v2_200V(1), <MPR Range>(1), <MPR ...",2022-09-26
25,22_09_30-11_25_07-DST-1_3_12_2_1107_5_2_34_18984,CAU002N028,"3D_localizer_1mm_200V(1), Quick_Tx_calib_200V_...",2022-09-30


In [14]:
# check whether retro recon is done on MP2RAGE
# this doesn't currently include those which were done by RR (not reuploaded to XNAT)
is_mp2rage_recon=bbb['Scans'].str.contains('MP2RAGE_UK7T_081018_tfl_wip944_b17stx_RR_UniformSens_MAG')
is_mp2rage_recon.value_counts()

True     34
False     5
Name: Scans, dtype: int64

In [16]:
# remaining scans have missing data.
bbb_needRR=bbb.loc[~is_mp2rage_recon]
bbb_needRR[['MR ID','Subject']]

,MR ID,Subject
5,22_06_17-11_24_30-DST-1_3_12_2_1107_5_2_34_18984,CAU002N006
18,22_08_15-11_08_13-DST-1_3_12_2_1107_5_2_34_18984,CAU002N019
24,22_09_26-11_09_05-DST-1_3_12_2_1107_5_2_34_18984,CAU002N027
25,22_09_30-11_25_07-DST-1_3_12_2_1107_5_2_34_18984,CAU002N028
26,22_10_03-09_51_15-DST-1_3_12_2_1107_5_2_34_18984,CAU002N029
